In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import glob
import numpy as np
import cv2

def load_train_lfw_data(data_dir):
    # Danh sách tất cả các tên thư mục (mỗi thư mục chứa các ảnh của một người)
    person_dirs = glob.glob(os.path.join(data_dir, '*'))

    images = []
    names = []

    for person_dir in person_dirs:
        person_name = os.path.basename(person_dir)  # Tên người
        image_paths = glob.glob(os.path.join(person_dir, '*.jpg'))

        for img_path in image_paths:
            img = cv2.imread(img_path)  # Đọc ảnh với 3 kênh màu (RGB)
            img = cv2.resize(img, (224, 224))  # Resize về kích thước mong muốn
            images.append(img)
            names.append(person_name)

    images = np.array(images)
    names = np.array(names)

    return images, names

data_dir = '/content/drive/MyDrive/Colab Notebooks/Train_ver3'
X, names = load_train_lfw_data(data_dir)

In [ ]:
# Encode nhãn thành số nguyên
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
names_encoded = le.fit_transform(names)

# Lưu encoder để dùng sau
import joblib
joblib.dump(le, '/content/drive/MyDrive/label_encoder_ver3.pkl')

['/content/drive/MyDrive/label_encoder_ver3.pkl']

In [ ]:
# Chia train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, names_encoded, test_size=0.2, stratify=names_encoded)

# Thông tin dữ liệu
print(X_train.shape, X_test.shape)

(772, 224, 224, 3) (194, 224, 224, 3)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20, #xoay ảnh tối đa 20 độ
    width_shift_range=0.2, #dịch chuyển theo chiều rộng
    height_shift_range=0.2, #dịch chuyển theo chiều cao
    horizontal_flip=True, #Lật ảnh theo chiều ngang
    brightness_range=[0.8, 1.2] #Thay đổi độ sáng
)

datagen.fit(X_train)

NameError: name 'X_train' is not defined

In [ ]:
#Tạo dữ liệu augmented
X_augmented = []
y_augmented = []

#Số lượng ảnh augmented cho mỗi ảnh gốc
augmented_per_image = 2

print(f"Tạo {augmented_per_image} ảnh biến thể cho mỗi ảnh gốc...")

for i in range(len(X_train)):
    if i % 100 == 0:
        print(f"Đã xử lý {i}/{len(X_train)} ảnh...")

    # Thêm ảnh gốc vào tập augmented
    X_augmented.append(X_train[i])
    y_augmented.append(y_train[i])

    # Tạo các ảnh biến thể
    no_img = 0
    for x_batch in datagen.flow(np.expand_dims(X_train[i], axis=0), batch_size=1):
        X_augmented.append(x_batch[0])
        y_augmented.append(y_train[i])
        no_img += 1
        if no_img == augmented_per_image:
            break

Tạo 2 ảnh biến thể cho mỗi ảnh gốc...
Đã xử lý 0/772 ảnh...
Đã xử lý 100/772 ảnh...
Đã xử lý 200/772 ảnh...
Đã xử lý 300/772 ảnh...
Đã xử lý 400/772 ảnh...
Đã xử lý 500/772 ảnh...
Đã xử lý 600/772 ảnh...
Đã xử lý 700/772 ảnh...


In [ ]:
X_train_model = np.array(X_augmented)
y_train_model= np.array(y_augmented)

In [ ]:
print(len(X_train_model), len(y_train_model))

2316 2316


In [ ]:
print(X_train_model.shape)

(2316, 224, 224, 3)


In [ ]:
!pip install tensorflow==2.15.0
!pip install tensorflow-addons

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa

model = load_model(
    '/content/drive/MyDrive/facenet_ver4.h5',
    custom_objects={'TripletSemiHardLoss': tfa.losses.TripletSemiHardLoss}
)

In [ ]:
import tensorflow as tf
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),
    loss=tfa.losses.TripletSemiHardLoss()
)

In [ ]:
batch_size = 80
gen_train = tf.data.Dataset.from_tensor_slices((X_train_model, y_train_model)).shuffle(2048).batch(batch_size).repeat()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    '/content/drive/MyDrive/facenet_au.h5',
    monitor='loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

In [ ]:
history = model.fit(
    gen_train,
    steps_per_epoch=30,
    epochs=50,
    callbacks=[checkpoint_callback]
)

Epoch 1/50
30/30 [==============================] - ETA: 0s - loss: 0.5787 
Epoch 1: loss improved from inf to 0.57867, saving model to /content/drive/MyDrive/facenet_au.h5


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


30/30 [==============================] - 1557s 52s/step - loss: 0.5787
Epoch 2/50
30/30 [==============================] - ETA: 0s - loss: 0.4877 
Epoch 2: loss improved from 0.57867 to 0.48772, saving model to /content/drive/MyDrive/facenet_au.h5
30/30 [==============================] - 1530s 51s/step - loss: 0.4877
Epoch 3/50
30/30 [==============================] - ETA: 0s - loss: 0.4064 
Epoch 3: loss improved from 0.48772 to 0.40638, saving model to /content/drive/MyDrive/facenet_au.h5
30/30 [==============================] - 1539s 51s/step - loss: 0.4064
Epoch 4/50
30/30 [==============================] - ETA: 0s - loss: 0.3644 
Epoch 4: loss improved from 0.40638 to 0.36437, saving model to /content/drive/MyDrive/facenet_au.h5
30/30 [==============================] - 1529s 51s/step - loss: 0.3644
Epoch 5/50
30/30 [==============================] - ETA: 0s - loss: 0.3119 
Epoch 5: loss improved from 0.36437 to 0.31188, saving model to /content/drive/MyDrive/facenet_au.h5
30/30 [=

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa

model = load_model(
    '/content/drive/MyDrive/facenet_au.h5',
    custom_objects={'TripletSemiHardLoss': tfa.losses.TripletSemiHardLoss}
)

In [ ]:
import tensorflow as tf
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),
    loss=tfa.losses.TripletSemiHardLoss()
)

In [ ]:
batch_size = 80
gen_train = tf.data.Dataset.from_tensor_slices((X_train_model, y_train_model)).shuffle(2048).batch(batch_size).repeat()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    '/content/drive/MyDrive/facenet_best.h5',
    monitor='loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

In [ ]:
history = model.fit(
    gen_train,
    steps_per_epoch=30,
    epochs=50,
    callbacks=[checkpoint_callback]
)

Epoch 1/50
30/30 [==============================] - ETA: 0s - loss: 0.2692 
Epoch 1: loss improved from inf to 0.26923, saving model to /content/drive/MyDrive/facenet_best.h5


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


30/30 [==============================] - 1557s 52s/step - loss: 0.2692
Epoch 2/50
30/30 [==============================] - ETA: 0s - loss: 0.1912 
Epoch 2: loss improved from 0.26923 to 0.19123, saving model to /content/drive/MyDrive/facenet_best.h5
30/30 [==============================] - 1561s 52s/step - loss: 0.1912
Epoch 3/50
30/30 [==============================] - ETA: 0s - loss: 0.1421 
Epoch 3: loss improved from 0.19123 to 0.14213, saving model to /content/drive/MyDrive/facenet_best.h5
30/30 [==============================] - 1552s 52s/step - loss: 0.1421
Epoch 4/50
30/30 [==============================] - ETA: 0s - loss: 0.0970 
Epoch 4: loss improved from 0.14213 to 0.09698, saving model to /content/drive/MyDrive/facenet_best.h5
30/30 [==============================] - 1541s 51s/step - loss: 0.0970
Epoch 5/50
30/30 [==============================] - ETA: 0s - loss: 0.0816 
Epoch 5: loss improved from 0.09698 to 0.08160, saving model to /content/drive/MyDrive/facenet_best.h5


#Test

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa

model = load_model(
    '/content/drive/MyDrive/facenet_best.h5',
    custom_objects={'TripletSemiHardLoss': tfa.losses.TripletSemiHardLoss}
)

In [ ]:
# Trích xuất embedding cho tập train và test
X_train_vecs = model.predict(X_train)
X_test_vecs = model.predict(X_test)

7/7 [==============================] - 140s 19s/step


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def _most_similarity(train_vecs, test_vec, train_labels):
    sims = cosine_similarity(train_vecs, test_vec)  # shape: (n_train, 1)
    idx = np.argmax(sims)
    return train_labels[idx]

In [ ]:
from sklearn.metrics import accuracy_score

y_preds = []
for vec in X_test_vecs:
    vec = vec.reshape(1, -1)
    y_pred = _most_similarity(X_train_vecs, vec, y_train)
    y_preds.append(y_pred)

print("Accuracy:", accuracy_score(y_test, y_preds))

Accuracy: 1.0


#Test trên LFW

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa

model = load_model(
    '/content/drive/MyDrive/facenet_best.h5',
    custom_objects={'TripletSemiHardLoss': tfa.losses.TripletSemiHardLoss}
)

In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/Colab Notebooks/lfw.zip"
extract_to = "/content/drive/MyDrive/Colab Notebooks/lfw"

# Tạo thư mục giải nén nếu chưa có
os.makedirs(extract_to, exist_ok=True)

# Giải nén
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Đã giải nén thành công vào:", extract_to)


✅ Đã giải nén thành công vào: /content/drive/MyDrive/Colab Notebooks/lfw


In [ ]:
import pandas as pd
import os

img_dir = "/content/drive/MyDrive/Colab Notebooks/lfw/lfw-deepfunneled/lfw-deepfunneled"
pairs_file = "/content/drive/MyDrive/Colab Notebooks/lfw/pairs.csv"

df = pd.read_csv(pairs_file, usecols=['name', 'imagenum1', 'imagenum2', 'Unnamed: 3'])

positive_pairs = df.iloc[:300]
negative_pairs = df.iloc[300:600]

pairs = []

# Positive pairs
for _, row in positive_pairs.iterrows():
    name = row['name']
    img1 = f"{img_dir}/{name}/{name}_{int(row['imagenum1']):04d}.jpg"
    img2 = f"{img_dir}/{name}/{name}_{int(row['imagenum2']):04d}.jpg"
    pairs.append((img1, img2, 1))

# Negative pairs
for _, row in negative_pairs.iterrows():
    name1 = row['name']
    img1 = f"{img_dir}/{name1}/{name1}_{int(row['imagenum1']):04d}.jpg"
    name2 = row['imagenum2']
    img2 = f"{img_dir}/{name2}/{name2}_{int(row['Unnamed: 3']):04d}.jpg"
    pairs.append((img1, img2, 0))

In [ ]:
import cv2
def preprocess_image(img_path):
    img = cv2.imread(img_path)  # Mặc định là BGR
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Đổi sang RGB
    img = cv2.resize(img, (224, 224))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)  # shape: (1, 224, 224, 3)
    return img

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

def evaluate_model(model, pairs, threshold=0.54):
    correct = 0
    total = len(pairs)
    for img1_path, img2_path, label in tqdm(pairs, desc="Evaluating custom model"):
        try:
            img1 = preprocess_image(img1_path)
            img2 = preprocess_image(img2_path)

            vec1 = model.predict(img1)[0]
            vec2 = model.predict(img2)[0]

            sim = cosine_similarity([vec1], [vec2])[0][0]
            pred = 1 if sim >= threshold else 0

            if pred == label:
                correct += 1
        except Exception as e:
            print(f"Lỗi xử lý cặp ảnh {img1_path} và {img2_path}: {e}")
    acc = correct / total
    return acc


In [ ]:
acc = evaluate_model(model, pairs, threshold=0.54)
print(f"\n Accuracy: {acc:.4f}")

Evaluating custom model:   0%|          | 0/600 [00:00<?, ?it/s]

1/1 [==============================] - 0s 267ms/step


Evaluating custom model:   0%|          | 1/600 [00:00<06:23,  1.56it/s]

1/1 [==============================] - 0s 265ms/step


Evaluating custom model:   0%|          | 2/600 [00:01<06:23,  1.56it/s]

...

1/1 [==============================] - 0s 304ms/step


Evaluating custom model: 100%|█████████▉| 599/600 [07:18<00:00,  1.45it/s]

1/1 [==============================] - 0s 466ms/step


Evaluating custom model: 100%|██████████| 600/600 [07:19<00:00,  1.36it/s]


 Accuracy: 0.7117


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/lfw/lfw-deepfunneled/lfw-deepfunneled/Tom_Ridge/Tom_Ridge_0016.jpg"
print(os.path.exists(path))


True
